In [1]:
import torch
print(f"GPU 是否可用: {torch.cuda.is_available()}")
print(f"当前 GPU: {torch.cuda.get_device_name(0)}")

GPU 是否可用: True
当前 GPU: NVIDIA A40


In [ ]:
import torch
import time

# 设置网格大小 (根据 PPT 第3页的示例)
N, M = 1000, 1000 
num_nodes = N * M

# 确保在 A40 GPU 上运行
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def build_poisson_sparse_matrix(N, M):
    print(f"正在构建 {N}x{M} 的系数矩阵...")
    indices = []
    values = []

    for i in range(N):
        for j in range(M):
            row = i * M + j
            
            # 中心点 (Diagonal): 系数为 4
            indices.append([row, row])
            values.append(4.0)
            
            # 5点模板的邻居点 (系数为 -1)
            for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < N and 0 <= nj < M:
                    col = ni * M + nj
                    indices.append([row, col])
                    values.append(-1.0)

    # 转换为 PyTorch 稀疏张量 (COO 格式)
    i_ts = torch.LongTensor(indices).t()
    v_ts = torch.FloatTensor(values)
    A = torch.sparse_coo_tensor(i_ts, v_ts, (num_nodes, num_nodes)).to(device)
    return A

# 1. 运行基准测试
start_time = time.time()
A_sparse = build_poisson_sparse_matrix(N, M)
print(f"矩阵构建完成，耗时: {time.time() - start_time:.2f} 秒")

# 2. 创建随机源项 b 和初始解 x
b = torch.randn(num_nodes, 1).to(device)
x = torch.zeros(num_nodes, 1).to(device)

# 3. 测试一次矩阵-向量乘法 (SpMV)
start_time = time.time()
r = torch.sparse.mm(A_sparse, x) - b
print(f"单次 SpMV 耗时: {time.time() - start_time:.5f} 秒")

正在构建 1000x1000 的系数矩阵...
矩阵构建完成，耗时: 4.29 秒
单次 SpMV 耗时: 0.00590 秒


In [6]:
import torch
import time

def run_jacobi_benchmark(N, M, iterations=100, device_type="cuda"):
    device = torch.device(device_type)
    num_nodes = N * M
    
    # 1. 准备数据
    # A_sparse 已经在你之前的单元格中构建好，这里假设它叫 A
    # 注意：Jacobi 迭代通常需要矩阵的对角线元素 D 和剩余部分 R
    # 在 5 点差分中，对角线 D 全是 4.0
    b = torch.ones((num_nodes, 1), device=device)
    x = torch.zeros((num_nodes, 1), device=device)
    D_inv = 1.0 / 4.0  # 泊松方程对角线逆
    
    # 构造迭代中的 A_off_diagonal (A - D)
    # 为了简化测试，我们直接用 A * x 并在迭代中处理
    
    torch.cuda.synchronize() if device_type == "cuda" else None
    start_time = time.time()
    
    for i in range(iterations):
        # Jacobi 公式: x^(k+1) = D^-1 * (b - R * x^k)
        # 其中 R*x = A*x - D*x = A*x - 4*x
        Ax = torch.sparse.mm(A_sparse.to(device), x)
        x = x + D_inv * (b - Ax)
        
        # 模拟每 10 次迭代打印一次进度，方便你在 Grafana 看到波动
        if (i+1) % 10 == 0:
            print(f"[{device_type}] 迭代次数: {i+1}/{iterations}")

    torch.cuda.synchronize() if device_type == "cuda" else None
    end_time = time.time()
    
    return end_time - start_time

# 执行对比测试
grid_size = 1000 # 对应 PPT 第3页
iters = 100

print(f"开始 {grid_size}x{grid_size} 网格的基准测试...")

# GPU 测试
gpu_time = run_jacobi_benchmark(grid_size, grid_size, iterations=iters, device_type="cuda")
print(f"--- GPU (A40) 总耗时: {gpu_time:.4f} 秒 ---")

# CPU 测试 (警告：可能会比 GPU 慢很多)
cpu_time = run_jacobi_benchmark(grid_size, grid_size, iterations=iters, device_type="cpu")
print(f"--- CPU 总耗时: {cpu_time:.4f} 秒 ---")

print(f"\n加速比 (Speedup): {cpu_time / gpu_time:.2f}x")

开始 1000x1000 网格的基准测试...
[cuda] 迭代次数: 10/100
[cuda] 迭代次数: 20/100
[cuda] 迭代次数: 30/100
[cuda] 迭代次数: 40/100
[cuda] 迭代次数: 50/100
[cuda] 迭代次数: 60/100
[cuda] 迭代次数: 70/100
[cuda] 迭代次数: 80/100
[cuda] 迭代次数: 90/100
[cuda] 迭代次数: 100/100
--- GPU (A40) 总耗时: 0.8605 秒 ---
[cpu] 迭代次数: 10/100
[cpu] 迭代次数: 20/100
[cpu] 迭代次数: 30/100
[cpu] 迭代次数: 40/100
[cpu] 迭代次数: 50/100
[cpu] 迭代次数: 60/100
[cpu] 迭代次数: 70/100
[cpu] 迭代次数: 80/100
[cpu] 迭代次数: 90/100
[cpu] 迭代次数: 100/100
--- CPU 总耗时: 13.6922 秒 ---

加速比 (Speedup): 15.91x


In [10]:
import torch
import time
import matplotlib.pyplot as plt
import numpy as np

# ==========================================
# 1. 核心功能函数定义
# ==========================================

def build_poisson_sparse_matrix(N, M, device):
    """
    构建 2D 泊松方程的 5 点差分系数矩阵 (COO 格式) [cite: 638, 657]
    """
    num_nodes = N * M
    indices = []
    values = []

    # 遍历网格构建 5 点模板 [cite: 696]
    for i in range(N):
        for j in range(M):
            row = i * M + j
            # 中心点 (Diagonal): 系数为 4
            indices.append([row, row])
            values.append(4.0)
            
            # 上下左右邻居 (Off-diagonal): 系数为 -1
            for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < N and 0 <= nj < M:
                    col = ni * M + nj
                    indices.append([row, col])
                    values.append(-1.0)

    # 转换为 PyTorch 稀疏张量 [cite: 654]
    i_ts = torch.LongTensor(indices).t()
    v_ts = torch.FloatTensor(values)
    # 将矩阵直接放置在目标设备上 
    A = torch.sparse_coo_tensor(i_ts, v_ts, (num_nodes, num_nodes)).to(device)
    return A

def run_jacobi_benchmark(A_sparse, N, M, iterations=100, device_type="cuda"):
    """
    执行 Jacobi 迭代基准测试 [cite: 663, 751]
    """
    device = torch.device(device_type)
    num_nodes = N * M
    
    # 准备向量数据
    b = torch.ones((num_nodes, 1), device=device)
    x = torch.zeros((num_nodes, 1), device=device)
    D_inv = 0.25  # 对于泊松 5 点模板，对角线元素的逆为 1/4
    
    # 同步并开始计时 (确保 GPU 计时准确) 
    if device_type == "cuda":
        torch.cuda.synchronize()
    
    start_time = time.time()
    
    for i in range(iterations):
        # 核心操作：矩阵-向量乘法 (SpMV) [cite: 661]
        # Jacobi 公式简化版: x_new = x_old + D_inv * (b - A * x_old)
        Ax = torch.sparse.mm(A_sparse, x)
        x = x + D_inv * (b - Ax)

    if device_type == "cuda":
        torch.cuda.synchronize()
        
    return time.time() - start_time

# ==========================================
# 2. 自动化实验逻辑 (Scaling Test)
# ==========================================

# 实验配置 
grid_sizes = [500, 1000, 1500, 2000] # 测试从 500^2 到 2000^2 的规模
iters = 100  # 每次测试迭代 100 次
gpu_results = []
cpu_results = []

print(f"开始集成基准测试 (使用设备: NVIDIA A40)...")

for n in grid_sizes:
    print(f"\n>>> 正在测试网格: {n}x{n} (总节点数: {n*n/1e6:.1f} Million)")
    
    # GPU 测试
    gpu_device = torch.device("cuda")
    A_gpu = build_poisson_sparse_matrix(n, n, gpu_device)
    t_gpu = run_jacobi_benchmark(A_gpu, n, n, iterations=iters, device_type="cuda")
    gpu_results.append(t_gpu)
    print(f"GPU (A40) 耗时: {t_gpu:.4f} 秒")
    
    # CPU 测试 (因 CPU 较慢，超过 1500 规模建议跳过或减少次数)
    #if n <= 1000:
    cpu_device = torch.device("cpu")
    A_cpu = build_poisson_sparse_matrix(n, n, cpu_device)
    t_cpu = run_jacobi_benchmark(A_cpu, n, n, iterations=iters, device_type="cpu")
    cpu_results.append(t_cpu)
    print(f"CPU 耗时: {t_cpu:.4f} 秒 | 加速比: {t_cpu/t_gpu:.2f}x")
    #else:
        #print("规模较大，跳过 CPU 基准测试以节省资源。")
        #cpu_results.append(None)

# ==========================================
# 3. 结果可视化 (Visualization)
# ==========================================

plt.figure(figsize=(10, 6))
plt.plot(grid_sizes, gpu_results, 'o-', linewidth=2, label='PyTorch GPU (A40)')
# 过滤掉为 None 的 CPU 数据进行绘图
valid_cpu_idx = [i for i, v in enumerate(cpu_results) if v is not None]
if valid_cpu_idx:
    plt.plot([grid_sizes[i] for i in valid_cpu_idx], 
             [cpu_results[i] for i in valid_cpu_idx], 
             's--', linewidth=2, label='PyTorch CPU')

plt.title('Poisson Solver Performance Scaling (Grid size vs Time) [cite: 664]')
plt.xlabel('Grid Side Dimension (N)')
plt.ylabel(f'Execution Time for {iters} Iterations (s)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.yscale('log') # 建议使用对数坐标以便观察巨大的性能差距
plt.show()

print("\n实验完成。请记录上述数据用于 Week 4 的报告编写。")

开始集成基准测试 (使用设备: NVIDIA A40)...

>>> 正在测试网格: 500x500 (总节点数: 0.2 Million)
GPU (A40) 耗时: 0.2158 秒
CPU 耗时: 2.8938 秒 | 加速比: 13.41x

>>> 正在测试网格: 1000x1000 (总节点数: 1.0 Million)
GPU (A40) 耗时: 0.8607 秒
CPU 耗时: 11.4994 秒 | 加速比: 13.36x

>>> 正在测试网格: 1500x1500 (总节点数: 2.2 Million)
GPU (A40) 耗时: 1.9799 秒
CPU 耗时: 25.8712 秒 | 加速比: 13.07x

>>> 正在测试网格: 2000x2000 (总节点数: 4.0 Million)
GPU (A40) 耗时: 3.6191 秒
CPU 耗时: 45.9797 秒 | 加速比: 12.70x

实验完成。请记录上述数据用于 Week 4 的报告编写。
